# Coursera Data Science - Capstone Project

In [258]:
import pandas   as pd
import numpy    as np
import urllib.request  as request

import json                               # library to handle JSON files
import matplotlib.cm     as cm            # Matplotlib plotting modules
import matplotlib.colors as colors
import folium # map rendering library
import re

from sklearn.preprocessing import StandardScaler
from geopy.geocoders       import Nominatim     # convert an address into latitude and longitude values
from pandas.io.json        import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster       import KMeans         # import k-means from clustering stage
from os import path


## Get Source Data
### The file is store in GIT hub
#### You can find more Australia Statistic Data from the following URL:
##### https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1410.02013-18?OpenDocument

In [259]:
# Load OZ_Population.xls (Population and People, ASGS, 2011 to 2018 )

url_src  = 'https://www.abs.gov.au/AUSSTATS/subscriber.nsf/log?openagent&14100ds0001_2011-18.xls&1410.0&Data%20Cubes&A104AABC499358F0CA2583FC00186495&0&2013-18&17.05.2019&Latest'
file_src = 'OZ_Population.xls'

if not path.exists( file_src ):
   print('Please wait, downloading {} from ABS .....'.format(file_src))
   request.urlretrieve(url_src, file_src)  

print('Loading {} .....'.format(file_src))
data_src1 = pd.read_excel(url_src, skiprows=7)
print('Load {} is complete with {} rows!\n'.format(file_src, data_src1.shape[0]))

Loading OZ_Population.xls .....
Load OZ_Population.xls is complete with 21679 rows!



In [260]:
# Load OZ_Edu_Empl.xls (Education and Employment, ASGS, 2011 to 2018)

url_src  = 'https://www.abs.gov.au/AUSSTATS/subscriber.nsf/log?openagent&14100ds0007_2011-18.xls&1410.0&Data%20Cubes&EE750E055289F56CCA2583FC00186A66&0&2013-18&17.05.2019&Latest'
file_src = 'OZ_Edu_Empl.xls'

if not path.exists( file_src ):
   print('Please wait, downloading {} from ABS .....'.format(file_src))
   request.urlretrieve(url_src, file_src)  

print('Loading {} .....'.format(file_src))
data_src2 = pd.read_excel(file_src,skiprows=6)
print('Load {} is complete with {} rows!\n'.format(file_src, data_src2.shape[0]))
data_src2.tail(9).tail(2)

Loading OZ_Edu_Empl.xls .....
Load OZ_Edu_Empl.xls is complete with 18803 rows!



,Unnamed: 0,Unnamed: 1,Unnamed: 2,4 year olds enrolled in preschool or in a preschool program,5 year olds enrolled in preschool or in a preschool program,Enrolled in preschool,Enrolled in a preschool program within a long day care centre,Children enrolled across more than one provider type,Total enrolled in a preschool program,Less than 15 hours,...,Number of Employee Jobs - Public administration and safety,Number of Employee Jobs - Education and training,Number of Employee Jobs - Health care and social assistance,Number of Employee Jobs - Arts and recreation services,Number of Employee Jobs - Other services,Number of Employee Jobs - Total,Labour Force,Unemployed,Unemployment rate,Participation rate
18801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18802,© Commonwealth of Australia 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleansing for OZ_Population.xls

In [261]:

Col_Nbrs  = list(range(0,3)) + [72] + list(range(85, 106))
Col_Names = ['CODE','Suburb','YEAR','Median Age','Born_in_Oceania_Ex_OZ','Born_in_North_West_Europe','Born_in_Southern_Eastern_Europe','Born_in_North_Africa_Middle_East','Born_in_South_East_Asia','Born_in_North_East_Asia','Born_in_Southern_Central_Asia','Born_in_America','Born_in_Sub_Saharan_Africa','Born_Overseas','Relg_Buddhism','Relg_Christianity','Relg_Hinduism','Relg_Islam','Relg_Judaism','Relg_Other','Relg_Secular','Relg_not_stated','Residency_Citizen','Residency_not_Citizen','Residency_not_Stated']

data1 = data_src1.iloc[:, Col_Nbrs]
data1.columns = Col_Names


In [262]:
data_src1.tail(10).head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,0-14 years,15-24 years,25-34 years,35-44 years,45-54 years,55-64 years,65-74 years,...,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
21669,901031003,Jervis Bay,2018,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
21670,901041004,Norfolk Island,2016,17.2,6.3,6.7,13,15.4,17.6,14.2,...,0.3,-,-,0.4,27.3,9.5,81.5,11.6,6.9,48.7
21671,901041004,Norfolk Island,2017,16.7,7.4,6.2,12.2,14.8,18.3,15,...,-,-,-,-,-,-,-,-,-,-


In [263]:
code_num  = data1.CODE.apply(pd.to_numeric, errors='coerce')
code_filt = (code_num > 200000000) & (code_num < 300000000)   # Melbourne Data
data_pop  = data1[code_filt].reset_index(drop=True)
print(data_pop.shape)
data_pop.tail(11).head(2)

(3665, 25)


,CODE,Suburb,YEAR,Median Age,Born_in_Oceania_Ex_OZ,Born_in_North_West_Europe,Born_in_Southern_Eastern_Europe,Born_in_North_Africa_Middle_East,Born_in_South_East_Asia,Born_in_North_East_Asia,...,Relg_Christianity,Relg_Hinduism,Relg_Islam,Relg_Judaism,Relg_Other,Relg_Secular,Relg_not_stated,Residency_Citizen,Residency_not_Citizen,Residency_not_Stated
3654,217041479,Warrnambool - North,2016,38,1.3,2.7,0.4,0.2,0.7,1.1,...,57.5,0.3,0.2,-,0.2,31.5,9.6,89.4,3.9,6.7
3655,217041479,Warrnambool - North,2017,38.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


## Data Cleansing for OZ_Edu_Empl.xls

In [264]:
Col_Names = ['CODE','YEAR','Edu_PostGraduate','Edu_Diploma','Edu_Bachelor','Edu_Adv_Dipl','Unempl_Rate']
Col_Nbrs  = [data_src2.columns[0],data_src2.columns[2],'Postgraduate Degree','Graduate Diploma, Graduate Certificate','Bachelor Degree','Advanced Diploma, or Diploma ','Unemployment rate']

data2 = data_src2[Col_Nbrs]
data2.columns = Col_Names
data2.shape

(18803, 7)

In [265]:
code_num  = data2.CODE.apply(pd.to_numeric, errors='coerce')
code_filt = (code_num > 20000000) & (code_num < 300000000)   # Melbourne Data
data_edu  = data2[code_filt].reset_index(drop=True)
print(data_edu.shape)
data_edu.head(3)

(7184, 7)


,CODE,YEAR,Edu_PostGraduate,Edu_Diploma,Edu_Bachelor,Edu_Adv_Dipl,Unempl_Rate
0,101021007,2011,4.4,2.7,12.3,7.7,4
1,101021007,2012,-,-,-,-,-
2,101021007,2013,-,-,-,-,-


## Merge OZ_Population.xls and OZ_Edu_Empl.xls into a single table
###  Use Year 2016 as it has the most complete data

In [266]:
data_merged_tmp = data_pop.join(data_edu.set_index(['CODE','YEAR']), on=['CODE','YEAR'], how='left')

data = data_merged_tmp[ data_merged_tmp['YEAR'] == 2016]
data.reset_index(drop=True,inplace=True)

#### Replace all missing values with Zeroes #####
data.iloc[:,3:].replace(to_replace = '-', value = 0, inplace=True)

print('Number of Rows After Merge  :', data_merged_tmp.shape[0])
print('Number of Rows For Year 2016:', data.shape[0])
data.head()

Number of Rows After Merge  : 3665
Number of Rows For Year 2016: 459


,CODE,Suburb,YEAR,Median Age,Born_in_Oceania_Ex_OZ,Born_in_North_West_Europe,Born_in_Southern_Eastern_Europe,Born_in_North_Africa_Middle_East,Born_in_South_East_Asia,Born_in_North_East_Asia,...,Relg_Secular,Relg_not_stated,Residency_Citizen,Residency_not_Citizen,Residency_not_Stated,Edu_PostGraduate,Edu_Diploma,Edu_Bachelor,Edu_Adv_Dipl,Unempl_Rate
0,201011001,Alfredton,2016,34.1,1.1,3.1,0.8,0.4,1.4,1.4,...,34.7,7.5,90,5.3,4.8,4.2,3.1,15.3,9.4,5.1
1,201011002,Ballarat,2016,41.5,1.2,4,0.8,0.3,1,1.1,...,35.7,9.3,88.5,4.8,6.7,6.2,4.3,19.4,9.8,6.5
2,201011003,Ballarat - North,2016,38.7,0.7,3.6,0.8,0.2,0.8,0.4,...,36.5,8.9,90.7,3.1,6.2,3.6,3.3,14.4,8.7,6.6
3,201011004,Ballarat - South,2016,35.3,0.9,3.7,0.7,0.5,0.9,1.1,...,40.1,10,88.1,4.6,7.4,2.5,2,10.2,8,9
4,201011005,Buninyong,2016,38,0.9,5.5,0.7,0.3,0.5,1.4,...,40.6,9.4,89.4,4.1,6.4,5.3,3.7,16.9,10.1,6.1


## Set color scheme for the clusters

In [267]:
x = np.arange(n_clus)
ys = [i + x + (i*x)**2 for i in range(n_clus)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

## Get Latitude and Longitude of all suburbs

In [268]:
file_src = 'GeoLoc.csv'

#######
### Get Geo Location if the GeoLoc.csv does not exist ###
#######
if not path.exists( file_src ):
   print('Please wait, getting Geo Location from Nominatim .....')
   geolocator = Nominatim(user_agent="Capstone")
   
   GeoLoc = {}
   for suburb in data['Suburb']:
      sub_vic = suburb + ", Vic"
      loc = geolocator.geocode(sub_vic)
      if loc is not None:
         GeoLoc[suburb] = [loc.latitude, loc.longitude]
    
   GeoLoc_dict = pd.DataFrame.from_dict(GeoLoc, orient='index')
   GeoLoc_dict.columns = ['latitude', 'longitude']
   GeoLoc_dict.to_csv('GeoLoc.csv')

GeoLoc = pd.read_csv(file_src, index_col=0)    
GeoLoc.head()

cbd_loc = geolocator.geocode("City of Melbourne, Vic")

## Define a Function to Get Columns based on the interest

In [269]:
def GetColumns ( p_interest ):
   
    cols = []
    for col in data_merged.columns:
        if re.match(p_interest + '.*', col) :
           cols = cols + [col]
    return cols

## Do Clustering based on Religion based

In [381]:
#cols = GetColumns('Relg')
cols = ['Relg_Buddhism', 'Relg_Christianity', 'Relg_Hinduism', 'Relg_Islam', 'Relg_Judaism',]

In [382]:
religion_df = data[cols]
religion_df = religion_df.astype(float)

In [383]:
religion_df.head()

,Relg_Buddhism,Relg_Christianity,Relg_Hinduism,Relg_Islam,Relg_Judaism
0,1.2,53.6,1.5,0.8,0.1
1,0.9,52.0,0.8,0.5,0.1
2,0.8,52.4,0.4,0.3,0.1
3,0.8,47.2,0.7,0.6,0.0
4,0.4,48.7,0.2,0.3,0.0


### Normalized the data

In [384]:
religion_scalar = StandardScaler().fit_transform( religion_df )
religion_scalar

array([[-0.42082813,  0.51561707, -0.14814156, -0.38056912, -0.1597414 ],
       [-0.50792874,  0.35711176, -0.44359706, -0.44033006, -0.1597414 ],
       [-0.53696228,  0.39673809, -0.61242878, -0.48017069, -0.1597414 ],
       ...,
       [-0.62406288,  0.90197375, -0.73905256, -0.52001132, -0.19398769],
       [-0.56599581,  0.90197375, -0.65463671, -0.500091  , -0.19398769],
       [-0.53696228,  0.42645783, -0.61242878, -0.48017069, -0.19398769]])

In [390]:
religion_scalar.min()

-4.794310668214761

In [392]:
religion_scalar.max()

13.881236614213263

### Clustering

In [385]:
n_clus = 5
religion_k_means = KMeans(init = "k-means++", n_clusters = n_clus, n_init = 12)

In [386]:
religion_k_means.fit( religion_scalar )

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=12, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [377]:
print( religion_k_means.labels_ )
print( len( religion_k_means.labels_ ) )

[0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 0 0 3 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 3 3 0 0 0 0 0
 0 0 0 0 0 3 0 0 0 0 0 0 3 0 0 0 3 0 0 0 0 0 0 0 0 0 3 0 0 3 3 3 0 0 3 3 3
 0 0 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0 0 0
 0 3 3 0 0 0 0 0 0 0 0 0 0 0 3 0 3 0 0 0 0 0 0 0 0 3 3 2 2 3 0 0 3 3 0 0 3
 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 1 1 3 0 0 3 3 3 0 3 0 1 0 1 0 0 1
 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 1 3 1 0 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 3 0 0 0 0
 0 3 0 3 0 3 0 0 3 0 0 0 0 0 1 1 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1
 1 1 1 1 1 3 1 1 1 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1 3 3 3
 3 3 0 1 0 0 0 3 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 3 0 0 0 0 0 0 3 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
459


In [387]:
### cbd_loc has been set above ### cbd_loc = geolocator.geocode("City of Melbourne, Vic")

map_relg = folium.Map(location=[cbd_loc.latitude, cbd_loc.longitude], zoom_start=11)

for i in range( len( religion_k_means.labels_ ) ):
   suburb  = data.loc[i,['Suburb']][0]
   
   if suburb not in GeoLoc.index:
      continue
    
   cluster = religion_k_means.labels_[i]
   colour  = rainbow[cluster] 
   # print( [GeoLoc.loc[suburb][0], GeoLoc.loc[suburb][1]] )
   
   folium.CircleMarker( [GeoLoc.loc[suburb][0], GeoLoc.loc[suburb][1]],
        radius=5, color=colour, fill=True, fill_color=colour,
        popup=suburb,
        fill_opacity=0.7 ).add_to(map_relg) 


In [388]:
map_relg

In [389]:
for i in data[ ['Suburb','Relg_Judaism'] ].sort_values(by = 'Relg_Judaism', ascending=False )['Suburb'].head(10).index:
    print(i, data.loc[i,'Suburb'], religion_k_means.labels_[i])

175 Caulfield - North 2
176 Caulfield - South 2
177 Elsternwick 4
132 St Kilda East 4
181 Bentleigh East (North) 4
136 Toorak 4
180 Ormond - Glen Huntly 0
169 Brighton East 4
182 Bentleigh East (South) 1
173 Bentleigh - McKinnon 4
